In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn import metrics
import matplotlib.pyplot as plt
from scipy import signal
import glob
import antropy
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from scipy.signal import butter, sosfilt, sosfreqz, detrend,stft
import math
from scipy.signal import stft,find_peaks
from scipy.stats import kurtosis,skew
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import pywt

#### extract wrong label data

In [6]:
label = pd.read_parquet(r"wrong_instances_rs_1.parquet")
label.head()

print(len(label[label["label_eating"] == 0]))
print(len(label[label["label_eating"] == 1]))

1369
1262


In [4]:
wrong_label = label

def merge_overlapping_pairs(pairs):
    if not pairs:
        return pairs

    merged_pairs = [pairs[0]]
    for current_start, current_end in pairs[1:]:
        last_merged_start, last_merged_end = merged_pairs[-1]

        if (current_start <= last_merged_end or current_start == last_merged_end+1):
            merged_pairs[-1] = (last_merged_start, max(last_merged_end, current_end))
        else:
            merged_pairs.append((current_start, current_end))

    return merged_pairs

def sort_and_pair(x):
    sorted_x = x.sort_values(by='data_pt_start')
    unique_sorted_pairs = sorted(set(zip(sorted_x['data_pt_start'], sorted_x['data_pt_end'])),
                                 key=lambda pair: pair[0])
    return merge_overlapping_pairs(unique_sorted_pairs)

def read_offset(file_name):
    target_path = "label/"+file_name.split("\\")[1]+"/n_sensor_data.xlsx"
    sheet = pd.read_excel(target_path,sheet_name = file_name.split("\\")[-1][0])
    return sheet.iloc[11,3]


grouped_data = wrong_label.groupby(['File', 'classifier']).apply(sort_and_pair).to_dict()

sorted_data = sorted(grouped_data.items(), key=lambda item: len(item[1]), reverse=True)

for file_name, data_points in sorted_data:
    print(f"File: {file_name}")
    print(len(data_points))
    offset = read_offset(file_name)
    for start, end in data_points:
        print(f"Start: {start-offset}, End: {end-offset}")


File: ./original\8.1_launch_1\0_sensor_data.csv
20
Start: 324.0, End: 443.0
Start: 564.0, End: 863.0
Start: 1464.0, End: 1583.0
Start: 2184.0, End: 2303.0
Start: 3624.0, End: 3743.0
Start: 5184.0, End: 5303.0
Start: 6084.0, End: 6203.0
Start: 8604.0, End: 8723.0
Start: 9744.0, End: 9863.0
Start: 11544.0, End: 11903.0
Start: 12324.0, End: 12443.0
Start: 12924.0, End: 13043.0
Start: 14964.0, End: 15083.0
Start: 15324.0, End: 15443.0
Start: 16224.0, End: 16343.0
Start: 16884.0, End: 17003.0
Start: 17304.0, End: 17423.0
Start: 17904.0, End: 18023.0
Start: 18924.0, End: 19043.0
Start: 19284.0, End: 19403.0
File: ./original\7.25_launch\0_sensor_data.csv
18


KeyboardInterrupt: 

In [5]:
output_directory = 'wrong_label/'

output_file = output_directory+"number_of_error.csv"
with open(output_file, 'w') as f:
    f.write('file, # of error\n')  

    for name, data in sorted_data:
        f.write(name.split("\\")[-2]+"_"+name.split("\\")[-1]+f",{len(data)}\n")
        print(name)


./original\7.27_dinner\1_sensor_data.csv
./original\7.26_dinner\1_sensor_data.csv
./original\7.31_dinner\0_sensor_data.csv
./original\8.1_launch_1\0_sensor_data.csv
./original\7.29_laun_2\0_sensor_data.csv
./original\7.22_launch\0_sensor_data.csv
./original\7.26_launch\2_sensor_data.csv
./original\7.27_dinner\0_sensor_data.csv
./original\8.3_breakfast\0_sensor_data.csv
./original\7.22_launch\1_sensor_data.csv
./original\7.23_breakfast\2_sensor_data.csv
./original\7.24_dinner\2_sensor_data.csv
./original\7.26_dinner\0_sensor_data.csv
./original\7.26_dinner\2_sensor_data.csv
./original\7.26_launch\1_sensor_data.csv
./original\7.27_dinner\2_sensor_data.csv
./original\7.27_launch\0_sensor_data.csv
./original\7.27_launch\1_sensor_data.csv
./original\8.1_launch_1\1_sensor_data.csv
./original\8.3_breakfast\1_sensor_data.csv
./original\7.22_launch\3_sensor_data.csv
./original\7.25_breakfast\0_sensor_data.csv
./original\7.26_launch\0_sensor_data.csv
./original\7.29_breakfast_2\0_sensor_data.csv

In [6]:

for file_name, data_points in sorted_data:
    output_file = output_directory+file_name.split("\\")[-2]+"_"+file_name.split("\\")[-1]
    offset = read_offset(file_name)
    with open(output_file, 'w') as f:
        f.write('Start,End\n')  # Writing the header

        # Write each data point pair to the file
        for start, end in data_points:
            f.write(f"{start-offset},{end-offset}\n")

    print(f"Results for {file_name} saved to {output_file}")


Results for ./original\7.27_dinner\1_sensor_data.csv saved to wrong_label/7.27_dinner_1_sensor_data.csv
Results for ./original\7.26_dinner\1_sensor_data.csv saved to wrong_label/7.26_dinner_1_sensor_data.csv
Results for ./original\7.31_dinner\0_sensor_data.csv saved to wrong_label/7.31_dinner_0_sensor_data.csv
Results for ./original\8.1_launch_1\0_sensor_data.csv saved to wrong_label/8.1_launch_1_0_sensor_data.csv
Results for ./original\7.29_laun_2\0_sensor_data.csv saved to wrong_label/7.29_laun_2_0_sensor_data.csv
Results for ./original\7.22_launch\0_sensor_data.csv saved to wrong_label/7.22_launch_0_sensor_data.csv
Results for ./original\7.26_launch\2_sensor_data.csv saved to wrong_label/7.26_launch_2_sensor_data.csv
Results for ./original\7.27_dinner\0_sensor_data.csv saved to wrong_label/7.27_dinner_0_sensor_data.csv
Results for ./original\8.3_breakfast\0_sensor_data.csv saved to wrong_label/8.3_breakfast_0_sensor_data.csv
Results for ./original\7.22_launch\1_sensor_data.csv saved